In [1]:
import numpy as np
import pandas as pd
import local_conformal as lc
import itertools
import progressbar
import torch

In [2]:
import ipdb
import cProfile
import pstats

# Parameters


| | Training|Validation|CDE-Tune|Calibrate| Test |
|-|-|-|-|-|-| 
|Izbicki| 1n |1n|0|1n| 1n |
|Us| .75n|.75n|.75n|.75n| 1n|

In [3]:
n = np.array([100,200,1000, 5000, 10000], dtype = int)
n_sims = 20

In [4]:
def k_values_select(n, n_train):
    out = list(np.arange(2,5, dtype = int)) + \
            list(np.arange(6,10, dtype = int)[::2]) +\
            list(np.array([15])) 
    
    if n > 1000:
        out += list(np.arange(16,33, dtype = int)[::8])
        
    out = np.unique(np.array(out + [int(n_train/100)])) # Izbicki's number
    out.sort()
    out = list(out)
        
    return out

In [5]:
def n_gaussians_select(n):
    upper = np.max([20, np.floor(n**(1/3))])
    
    out = list(np.arange(1,5, dtype =int)) +\
        list(np.arange(6,11, dtype =int)[::2]) +\
        list(np.array([15, 20], dtype = int)) +\
        list(np.arange(20, upper, dtype = int)[::5])

    return out

In [6]:
def lr_select():
    out = list(1/10**(np.arange(2,7)))
    return out

In [7]:
def hidden_select(n):
    n_based_value = np.int(np.floor(n**.5))
    if (n_based_value > 10):
        upper = int(np.min([50, n_based_value]))
        out = list(np.arange(5, 10, dtype = int)) +\
            list(np.arange(10, 30, dtype = int)[::5]) +\
            list(np.arange(30,n_based_value, dtype =int)[::10])

    else:
        out = list(np.arange(5, 10, dtype = int))

    return out

In [8]:

#     def full_process(n, n_sims, sigma  = 4):
#         # get ranges of all parameters

#         # or write a python script, and run this in parallel (slurm style)?

#         all_info_across = pd.DataFrame(columns = ["n_sim",
#                                                "approach_type",
#                                                "mdn_n_gaussian",
#                                                "mdn_lr",
#                                                "mdn_hidden1",
#                                                "mdn_hidden2",
#                                                "mdn_train_error",
#                                                "mdn_val_error",
#                                                "q_lr",
#                                                "q_hidden1",
#                                                "q_hidden2",
#                                                "q_train_error",
#                                                "q_val_error",
#                                                "k",
#                                                "x_group",
#                                                "validity_truth",
#                                                "validity_test",
#                                                "efficiency_truth"
#                                                ])
#         for sim_idx in np.arange(n_sims):
#             data_all = lc.data_generation(n, sigma = sigma)
#             inner_info_iz = inner_process_izbicki(data_all, ...parameters) 
#             inner_info_us = inner_process_us(data_all, ...parameters) 

#             inner_info["n_sim"] = s_idx
#             inner_info2 = inner_info[["n_sim",
#                                       "approach_type",
#                                        "mdn_n_gaussian",
#                                        "mdn_lr",
#                                        "mdn_hidden1",
#                                        "mdn_hidden2",
#                                        "mdn_train_error",
#                                        "mdn_val_error",
#                                        "q_lr",
#                                        "q_hidden1",
#                                        "q_hidden2",
#                                        "q_train_error",
#                                        "q_val_error",
#                                        "k",
#                                        "x_group",
#                                        "validity_truth",
#                                        "validity_test",
#                                        "efficiency_truth"
#                                        ]]

#             all_info_across = all_info_across.append(inner_info2)

#         return all_info_across 

In [9]:
n = 200
data_all = lc.data_generation(n, 4)

data_izbicki_list = lc.stratified_data_splitting(data_all,
                                                prop_vec = np.array([1,1,0,1,1])/4)
x_train_iz,y_train_iz, g_train_iz = lc.torchify_data(data_izbicki_list[0].x, 
                                        data_izbicki_list[0].y, 
                                        data_izbicki_list[0].group_info)
x_val_iz,y_val_iz, g_val_iz = lc.torchify_data(data_izbicki_list[1].x, 
                                        data_izbicki_list[1].y, 
                                        data_izbicki_list[1].group_info)
x_cal_iz,y_cal_iz, g_val_iz = lc.torchify_data(data_izbicki_list[3].x, 
                                        data_izbicki_list[3].y, 
                                        data_izbicki_list[3].group_info)
x_test_iz,y_test_iz, g_test_iz = lc.torchify_data(data_izbicki_list[4].x, 
                                        data_izbicki_list[4].y, 
                                        data_izbicki_list[4].group_info)



column_order = [
               "approach_type",
               "mdn_n_gaussian",
               "mdn_lr",
               "mdn_hidden1",
               "mdn_hidden2",
               "mdn_train_error",
               "mdn_val_error",
               "q_lr",
               "q_hidden1",
               "q_hidden2",
               "q_train_error",
               "q_val_error",
               "k",
               "x_group",
               "validity_truth",
               "validity_test",
               "efficiency_truth"
               ]
all_info_out = pd.DataFrame(columns = column_order)

all_info_out


n_guassian, lr_1, hidden1_1, hidden2_1 = 1, 1e-03, 10, 10
epochs = 10000

In [10]:
mdn_model_iz, mdn_opt_iz, train_error_iz, val_error_iz = \
                                lc.tune_first_nn(x_train = x_train_iz,
                                                 y_train = y_train_iz,
                                                 x_val = x_val_iz,
                                                 y_val = y_val_iz,
                                                 epochs = epochs,
                                                 n_gaussians=n_guassian,
                                                 n_hidden1 = hidden1_1,
                                                 n_hidden2 = hidden2_1,
                                                 lr = lr_1, verbose = True)

|#####################################| (Time:  0:00:22, Elapsed Time: 0:00:22)


In [11]:
y_range = np.array([-25,25]).reshape((-1,1))
y_splits = 10000
t_range_size = 500

In [12]:
mdn_model_iz.prep_for_cde(y_range, y_splits)
model_y_range_iz = (mdn_model_iz.y_range[0],
                mdn_model_iz.y_range[1])
model_n_grid_iz = mdn_model_iz.n_grid
y_grid_iz = np.linspace(model_y_range_iz[0], model_y_range_iz[1],
                       model_n_grid_iz)

In [13]:
y_delta_iz = (mdn_model_iz.y_range[1] - mdn_model_iz.y_range[0]) /\
            mdn_model_iz.n_grid

cde_train_iz = mdn_model_iz.cde_predict_grid(x_train_iz).detach().numpy() 
cde_cal_iz = mdn_model_iz.cde_predict_grid(x_cal_iz).detach().numpy()
cde_test_iz = mdn_model_iz.cde_predict_grid(x_test_iz).detach().numpy()

t_grid = np.linspace(cde_train_iz.min(), cde_train_iz.max(), t_range_size)


profile_density_train_iz = lc.profile_density(cde_train_iz, 
                                           t_grid = t_grid,
                                           z_delta = y_delta_iz)

profile_density_cal_iz = lc.profile_density(cde_cal_iz,
                                         t_grid = t_grid,
                                         z_delta = y_delta_iz)

profile_density_test_iz = lc.profile_density(cde_test_iz,
                                         t_grid = t_grid,
                                         z_delta = y_delta_iz)

In [14]:
k = 3
quantiles = np.arange(1,500)/500

In [15]:
kmeans_model_iz, _, _ = lc.profile_grouping(profile_density_train_iz, 
                                            k = k)

_, _, grouping_cal_iz = lc.profile_grouping(profile_test = profile_density_cal_iz, Kmeans_model = kmeans_model_iz)
_, _, grouping_test_iz = lc.profile_grouping(profile_test = profile_density_test_iz, Kmeans_model = kmeans_model_iz)



In [16]:

# get distribution of conformal scores -------
cs_cal_iz = mdn_model_iz.cde_predict(x_cal_iz, y_cal_iz)

df_cs_group_cal_iz = pd.DataFrame(data = {"x": x_cal_iz.numpy().ravel(),
                                            "cs": cs_cal_iz.detach().numpy(),
                                            "grouping":grouping_cal_iz})


# will need to calculate cuts per each threshold per group

cs_test_iz = mdn_model_iz.cde_predict(x_test_iz, y_test_iz)


df_cs_group_test_iz = pd.DataFrame(data = {"x": x_test_iz.numpy().ravel(),
                                           "cs": cs_test_iz.detach().numpy(),
                                           "grouping":grouping_test_iz})

thresholds_mat_test_iz, _ = lc.thresholds_per_group(df_cs_group_cal_iz,
                                              desired_props = quantiles,
                                              append = df_cs_group_test_iz)



In [17]:
true_cde_test_iz = lc.true_cde_out(x_test_iz.numpy().ravel(), y_grid_iz)

In [18]:
import scipy.stats

In [19]:
true_cde_test_iz = lc.true_cde_out(x_test_iz.numpy().ravel(), y_grid_iz)

In [20]:
true_thresholds_test_iz = lc.true_thresholds_out(true_cde = true_cde_test_iz, 
                                                z_delta = y_delta_iz,
                                                expected_prop = quantiles)

In [21]:
test_bin_raw_iz = lc.difference_actual_validity(df_cs_group_test_iz,
                                           thresholds_mat_test_iz)

In [22]:
v_iz,v_raw_iz, e_iz = lc.difference_validity_and_efficiency(true_cde = true_cde_test_iz,
                                                 predict_grid = cde_test_iz,
                                                 true_grid = true_cde_test_iz,
                                                 thresholds_predict = thresholds_mat_test_iz,
                                                 thresholds_true = true_thresholds_test_iz,
                                                 df_cs_grouping = df_cs_group_test_iz,
                                                 true_grouping = g_test_iz.numpy().ravel(),
                                                 expected_prop = quantiles, 
                                                 z_delta = y_delta_iz,
                                                 verbose = True)


|#####################################| (Time:  0:00:54, Elapsed Time: 0:00:54)


In [23]:
def _combine_pivot_long_with_info(single_row_df, v_pl_df, v_raw_pl_df, e_pl_df):
    """
    helper function to add single_row_df info into new columns after
    rbind(v_pl_df, v_raw_pl_df, e_pl_df). We first also add a "type" column to
    these matrices to identify them
    
    Arguments:
    ----------
    single_row_df : pandas DF (1, p)
    v_pl_df : pands DF (n2, p2)
    v_raw_pl_df : pands DF (n3, p2)
    e_pl_df : pandas DF (n4, p2)
    
    Returns:
    --------
    combined pandas Data Frame (n2+n3+n4, p2+p)
    """
    v_iz2 = v_pl_df.copy()
    v_iz2["type"] = "validity_oracle"


    e_iz2 = e_pl_df.copy()
    e_iz2["type"] = "efficiency_oracle"

    v_raw_iz2 = v_raw_pl_df.copy()
    v_raw_iz2["type"] = "validity_test_set"
    
    info_combined = pd.concat([v_iz2.reset_index(drop = True), 
           v_raw_iz2.reset_index(drop = True), 
           e_iz2.reset_index(drop = True)], axis = 0)
    
    info_all = info_combined.copy()
    for c in single_row_df.columns:
        info_all[c] = np.array(single_row_df[c]).reshape(())

    col_names = single_row_df.columns.tolist() + info_combined.columns.tolist()

    info_all = info_all[col_names]
    
    return info_all


In [24]:
def inner_process_izbicki(data_all,
                         k_range, n_gaussians_range, lr_range, hidden_range, 
                          verbose = True, 
                          epochs = 10000, 
                          y_range = np.array([-200,200]).reshape((-1,1)),
                          y_splits = 10000,
                          quantiles = np.arange(1,500)/500,
                          conformal_quantiles = np.arange(1,500)/500,
                          t_range_size = 500):
    
    # data splitting ---------
    data_izbicki_list = lc.stratified_data_splitting(data_all,
                                                    prop_vec = np.array([1,1,0,1,1])/4)
    x_train_iz,y_train_iz, g_train_iz = lc.torchify_data(data_izbicki_list[0].x, 
                                            data_izbicki_list[0].y, 
                                            data_izbicki_list[0].group_info)
    x_val_iz,y_val_iz, g_val_iz = lc.torchify_data(data_izbicki_list[1].x, 
                                            data_izbicki_list[1].y, 
                                            data_izbicki_list[1].group_info)
    x_cal_iz,y_cal_iz, g_val_iz = lc.torchify_data(data_izbicki_list[3].x, 
                                            data_izbicki_list[3].y, 
                                            data_izbicki_list[3].group_info)
    x_test_iz,y_test_iz, g_test_iz = lc.torchify_data(data_izbicki_list[4].x, 
                                            data_izbicki_list[4].y, 
                                            data_izbicki_list[4].group_info)

    
    if verbose:
        bar = progressbar.ProgressBar()
        larger_iter = bar(list(itertools.product(n_gaussians_range, lr_range, hidden_range)))
    else:
        larger_iter = itertools.product(n_gaussians_range, lr_range, hidden_range)
    
    out_df_full = None
    
    y_grid_iz = np.linspace(y_range[0], y_range[1],
                                y_splits).ravel()
    y_delta_iz = (y_range[1]-y_range[0])/y_splits
    
    true_cde_test_iz = lc.true_cde_out(x_test_iz.numpy().ravel(), y_grid_iz)
    
    true_thresholds_test_iz = lc.true_thresholds_out(
                                    true_cde = true_cde_test_iz, 
                                    z_delta = y_delta_iz,
                                    expected_prop = conformal_quantiles)
    
    
    for n_guassian, lr_1, hidden_1 in larger_iter: # no k_range and only MDN parameters currently
        
        hidden1_1 = hidden_1
        hidden2_1 = hidden_1
        # izbicki --------
        mdn_model_iz, mdn_opt_iz, train_error_iz, val_error_iz = \
                                        lc.tune_first_nn(x_train = x_train_iz,
                                                         y_train = y_train_iz,
                                                         x_val = x_val_iz,
                                                         y_val = y_val_iz,
                                                         epochs = epochs,
                                                         n_gaussians = n_guassian,
                                                         n_hidden1 = hidden1_1,
                                                         n_hidden2 = hidden2_1,
                                                         lr = lr_1,
                                                         verbose = False)
        mdn_model_iz.prep_for_cde(y_range, y_splits)
        #model_y_range_iz = (mdn_model_iz.y_range[0],
        #                 mdn_model_iz.y_range[1])
        #model_n_grid_iz = mdn_model_iz.n_grid
        #y_grid_iz = np.linspace(model_y_range_iz[0], model_y_range_iz[1],
        #                        model_n_grid_iz)

        # izbicki clustering -----
        #y_delta_iz = (mdn_model_iz.y_range[1] - mdn_model_iz.y_range[0]) /\
        #            mdn_model_iz.n_grid

        cde_train_iz = mdn_model_iz.cde_predict_grid(x_train_iz).detach().numpy() 
        cde_cal_iz = mdn_model_iz.cde_predict_grid(x_cal_iz).detach().numpy()
        cde_test_iz = mdn_model_iz.cde_predict_grid(x_test_iz).detach().numpy()
        
        t_grid = np.linspace(cde_train_iz.min(), cde_train_iz.max(), t_range_size)


        profile_density_train_iz = lc.profile_density(cde_train_iz, 
                                                   t_grid = t_grid,
                                                   z_delta = y_delta_iz)

        profile_density_cal_iz = lc.profile_density(cde_cal_iz,
                                                 t_grid = t_grid,
                                                 z_delta = y_delta_iz)

        profile_density_test_iz = lc.profile_density(cde_test_iz,
                                                 t_grid = t_grid,
                                                 z_delta = y_delta_iz)
        
        
        for k_val in k_range:

            kmeans_model_iz, _, _ = lc.profile_grouping(profile_density_train_iz, 
                                                        k = k)
            _, _, grouping_cal_iz = lc.profile_grouping(profile_test = profile_density_cal_iz, 
                                                        Kmeans_model = kmeans_model_iz)
            _, _, grouping_test_iz = lc.profile_grouping(profile_test = profile_density_test_iz, 
                                                        Kmeans_model = kmeans_model_iz)



            # get distribution of conformal scores -------
            cs_cal_iz = mdn_model_iz.cde_predict(x_cal_iz, y_cal_iz)

            df_cs_group_cal_iz = pd.DataFrame(data = {"x": x_cal_iz.numpy().ravel(),
                                                      "cs": cs_cal_iz.detach().numpy(),
                                                      "grouping": grouping_cal_iz})


            # will need to calculate cuts per each threshold per group
            cs_test_iz = mdn_model_iz.cde_predict(x_test_iz, y_test_iz)
            
            df_cs_group_test_iz = pd.DataFrame(data = {"x": x_test_iz.numpy().ravel(),
                                                       "cs": cs_test_iz.detach().numpy(),
                                                       "grouping": grouping_test_iz})

            thresholds_mat_test_iz, _ = lc.thresholds_per_group(df_cs_group_cal_iz,
                                                          desired_props = conformal_quantiles,
                                                          append = df_cs_group_test_iz)


            #ipdb.set_trace()
            v_iz, v_raw_iz, e_iz = lc.difference_validity_and_efficiency(
                                         true_cde = true_cde_test_iz,
                                         predict_grid = cde_test_iz,
                                         true_grid = true_cde_test_iz,
                                         thresholds_predict = thresholds_mat_test_iz,
                                         thresholds_true = true_thresholds_test_iz,
                                         df_cs_grouping = df_cs_group_test_iz,
                                         true_grouping = g_test_iz.numpy().ravel(), 
                                         expected_prop = conformal_quantiles, 
                                         z_delta = y_delta_iz,
                                         verbose = False)
            
            info_left = pd.DataFrame(data = {"mdn_n_gaussian": np.array(["iz"]),
                                                   "mdn_lr": lr_1,
                                                   "mdn_hidden1": hidden1_1,
                                                   "mdn_hidden2": hidden2_1,
                                                   "mdn_train_error": train_error_iz.detach().numpy(),
                                                   "mdn_val_error": val_error_iz.detach().numpy(),
                                                    "q_lr": None,
                                                   "q_hidden1": None,
                                                   "q_hidden2": None,
                                                   "q_train_error": None,
                                                   "q_val_error": None,
                                                   "k": k_val})

            out = _combine_pivot_long_with_info(info_left, v_iz, v_raw_iz, e_iz)
            
            if out_df_full is None:
                out_df_full = out
            else:
                out_df_full = pd.concat([out_df_full.reset_index(drop = True), 
                                         out.reset_index(drop = True)],
                                       axis = 0)
        
        return out_df_full



In [74]:
n = 100
data_all = lc.data_generation(n)
n_all = data_all.shape[0]
n_train_iz = n_all*1/4
n_train_us = n_all*3/16

k_range_iz = k_values_select(n_all, n_train_iz)
k_range_us = k_values_select(n_all, n_train_us)

n_guassians_range = n_gaussians_select(n_all)
lr_range = lr_select()
hidden_range = hidden_select(n_all)


In [26]:
(len(n_guassians_range), len(lr_range), len(hidden_range), len(k_range_iz)), \
n_guassians_range, lr_range, hidden_range, k_range_iz


((9, 5, 10, 9),
 [1, 2, 3, 4, 6, 8, 10, 15, 20],
 [0.01, 0.001, 0.0001, 1e-05, 1e-06],
 [5, 6, 7, 8, 9, 10, 15, 20, 25, 30],
 [2, 3, 4, 6, 8, 15, 16, 24, 32])

In [27]:
len(list(itertools.product(n_guassians_range, lr_range, hidden_range)))

450

In [28]:
iz_out = inner_process_izbicki(data_all = data_all,
                      k_range = k_range_iz[0:2], 
                      n_gaussians_range = n_guassians_range[0:2], 
                      lr_range = lr_range[0:2], 
                      hidden_range = hidden_range[0:2], 
                      verbose = True, 
                      epochs = 10000, 
                      y_range = np.array([-200,200]).reshape((-1,1)),
                      y_splits = 10000,
                      quantiles = np.arange(1,500)/500,
                      conformal_quantiles = np.arange(1,500)/500,
                      t_range_size = 500)

N/A% (0 of 8) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

In [29]:
cProfile.run("""iz_out = inner_process_izbicki(data_all = data_all,
                      k_range = k_range_iz[0:2], 
                      n_gaussians_range = n_guassians_range[0:2], 
                      lr_range = lr_range[0:2], 
                      hidden_range = hidden_range[0:2], 
                      verbose = True, 
                      epochs = 10000, 
                      y_range = np.array([-25,25]).reshape((-1,1)),
                      y_splits = 10000,
                      quantiles = np.arange(1,100)/100,
                      conformal_quantiles = np.arange(1,100)/100,
                      t_range_size = 500)""", "profile_izbicki.txt")

N/A% (0 of 8) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

In [30]:
import pstats
p = pstats.Stats('profile_izbicki.txt')
p.sort_stats("cumtime").print_stats()

Fri Jun 18 00:27:59 2021    profile_izbicki.txt

         7829404 function calls (7717994 primitive calls) in 44.133 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     21/1    0.002    0.000   44.133   44.133 {built-in method builtins.exec}
        1    0.011    0.011   44.133   44.133 <string>:1(<module>)
        1    0.325    0.325   44.121   44.121 <ipython-input-24-73f68a1f6036>:1(inner_process_izbicki)
        1    0.391    0.391   19.163   19.163 /Users/benjaminleroy/Documents/CMU/research/CDE-conformal/local_conformal/pytorch_models_1d.py:202(tune_first_nn)
        2    4.091    2.046    9.646    4.823 /Users/benjaminleroy/Documents/CMU/research/CDE-conformal/local_conformal/validity_and_efficiency.py:6(difference_validity_and_efficiency)
        3    0.019    0.006    9.311    3.104 /Users/benjaminleroy/Documents/CMU/research/CDE-conformal/local_conformal/hpd_process.py:268(profile_density)
     1164    1.014   

     1478    0.001    0.000    0.004    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/dtypes/common.py:830(is_signed_integer_dtype)
        6    0.000    0.000    0.004    0.001 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/sklearn/cluster/_kmeans.py:509(_labels_inertia)
        8    0.000    0.000    0.004    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/util/_decorators.py:225(wrapper)
        8    0.000    0.000    0.004    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:3837(reindex)
        2    0.000    0.000    0.004    0.002 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:7088(join)
        2    0.000    0.000    0.004    0.002 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:7212(_join_compat)
      459    0.000    0.000    0.004    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/panda

        1    0.000    0.000    0.001    0.001 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/groupby/ops.py:129(_get_splitter)
      459    0.001    0.000    0.001    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/generic.py:219(_init_mgr)
        5    0.000    0.000    0.001    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/torch/nn/modules/linear.py:68(__init__)
       87    0.001    0.000    0.001    0.000 {pandas._libs.algos.take_2d_axis0_int64_int64}
      140    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(ravel)
       52    0.000    0.000    0.001    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py:1667(is_boolean)
      654    0.000    0.000    0.001    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/dtypes/common.py:179(ensure_python_int)
       24    0.000    0.000    0.001    0.000 <__array_function__ inte

      403    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/generic.py:238(attrs)
       40    0.000    0.000    0.000    0.000 {method 'random_sample' of 'numpy.random.mtrand.RandomState' objects}
      529    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py:638(dtype)
      899    0.000    0.000    0.000    0.000 {built-in method builtins.next}
       82    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/dtypes/cast.py:549(infer_dtype_from_scalar)
       52    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/internals/blocks.py:225(get_values)
       48    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/indexes/multi.py:1175(__len__)
      323    0.000    0.000    0.000    0.000 {built-in 

       70    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/numpy/core/multiarray.py:311(where)
      131    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2333(_all_dispatcher)
        2    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/algorithms.py:1276(wrapper)
       33    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/indexes/range.py:444(equals)
        6    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/torch/nn/modules/module.py:175(add_module)
        1    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/progressbar/bar.py:102(_handle_resize)
       20    0.000    0.000    0.000    0.000 {built-in method pandas._libs.lib.is_bool_array}
       10    0.000    

        6    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/series.py:3917(_needs_reindex_multi)
        2    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:508(_selected_obj)
        1    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/groupby/ops.py:227(levels)
        8    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/groupby/ops.py:233(<listcomp>)
        1    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/groupby/ops.py:882(__init__)
        2    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/site-packages/pandas/core/reshape/reshape.py:295(<listcomp>)
        3    0.000    0.000    0.000    0.000 /Users/benjaminleroy/miniconda3/lib/python3.6/sit

## Us

In [31]:
data_us_list = lc.stratified_data_splitting(data_all, 
                                            prop_vec = np.array([.75,.75,.75,.75,1])/4)

x_train_us,y_train_us, g_train_us = lc.torchify_data(data_us_list[0].x, 
                                    data_us_list[0].y, 
                                    data_us_list[0].group_info)
x_val_us,y_val_us, g_val_us = lc.torchify_data(data_us_list[1].x, 
                                    data_us_list[1].y, 
                                    data_us_list[1].group_info)
x_cde_cal_us, y_cde_cal_us, g_cde_cal_us = lc.torchify_data(data_us_list[2].x, 
                                    data_us_list[2].y, 
                                    data_us_list[2].group_info)
x_cal_us,y_cal_us, g_cal_us = lc.torchify_data(data_us_list[3].x, 
                                    data_us_list[3].y, 
                                    data_us_list[3].group_info)
x_test_us,y_test_us, g_test_us = lc.torchify_data(data_us_list[4].x, 
                                    data_us_list[4].y, 
                                    data_us_list[4].group_info)

In [32]:
epochs = 10000
lr_1, hidden1_1, hidden2_1 = 1e-03, 10, 10
y_range = np.array([-25,25]).reshape((-1,1))
y_splits = 10000

In [33]:
y_grid_us = np.linspace(y_range[0], y_range[1],
                            y_splits).ravel()
y_delta_us = (y_range[1]-y_range[0])/y_splits

## current loc

In [34]:
true_cde_test_us = lc.true_cde_out(x_test_us.numpy().ravel(), y_grid_us)
true_hpd_test_us = lc.hpd_grid(true_cde_test_us, y_delta_us)

In [36]:
conformal_quantiles = np.arange(1,500)/500
quantiles = np.arange(1,500)/500

In [37]:
# this is just the conformal scores since we're using HPD as the cs values
true_thresholds_test_us = np.repeat(conformal_quantiles.reshape((1,-1)),
                                    x_test_us.shape[0], axis = 0)

In [38]:
mdn_model_us, mdn_opt_us, train_error_us, val_error_us = lc.tune_first_nn(
                                                 x_train = x_train_us,
                                                 y_train = y_train_us,
                                                 x_val = x_val_us,
                                                 y_val = y_val_us,
                                                 epochs = epochs,
                                                 n_gaussians = n_guassian,
                                                 n_hidden1 = hidden1_1,
                                                 n_hidden2 = hidden2_1,
                                                 lr = lr_1, 
                                                 verbose = True)

|#####################################| (Time:  0:00:15, Elapsed Time: 0:00:15)


In [39]:
mdn_model_us.prep_for_cde(y_range, y_splits)
model_y_range_us = (mdn_model_us.y_range[0],
                 mdn_model_us.y_range[1])
model_n_grid_us = mdn_model_us.n_grid
y_grid_us = np.linspace(model_y_range_us[0], model_y_range_us[1], model_n_grid_us)

# prepping for second model -------
cde_train_us = mdn_model_us.cde_predict_grid(x_train_us).detach().numpy()
cde_val_us = mdn_model_us.cde_predict_grid(x_val_us).detach().numpy()

hpd_train_base_us = lc.hpd_coverage(cde_train_us, y_grid_us, y_train_us.numpy())
hpd_val_base_us = lc.hpd_coverage(cde_val_us, y_grid_us, y_val_us.numpy())

In [40]:
hpd_train_us = torch.from_numpy(hpd_train_base_us.reshape((-1,1)))
hpd_val_us = torch.from_numpy(hpd_val_base_us.reshape((-1,1)))

In [41]:
lr_2, hidden1_2, hidden2_2 = 1e-03, 10, 10

In [42]:
q_model, q_opt, q_train_error_us, q_val_error_us = lc.tune_second_nn(x_train = x_train_us,
                                                y_train = hpd_train_us,
                                                x_val = x_val_us,
                                                y_val = hpd_val_us,
                                                epochs = epochs,
                                                quantiles = quantiles,
                                                n_hidden1 = hidden1_2,
                                                n_hidden2 = hidden2_2,
                                                lr = lr_2,
                                                verbose = True)

|#####################################| (Time:  0:00:55, Elapsed Time: 0:00:55)


In [43]:
hpd_estimate_cal_us = q_model(x_cal_us).detach().numpy()
hpd_estimate_test_us = q_model(x_test_us).detach().numpy()

In [44]:
k_val = 3

In [45]:
kmeans_model_us, grouping_cal_us, _ = \
    lc.profile_grouping(profile_train = hpd_estimate_cal_us, k = k_val)

In [46]:
_, _, grouping_test_us = lc.profile_grouping(profile_test = hpd_estimate_test_us, 
                                             Kmeans_model = kmeans_model_us)

In [47]:
cde_cal_us = mdn_model_us.cde_predict_grid(x_cal_us)
cde_test_us = mdn_model_us.cde_predict_grid(x_test_us)

In [48]:
hpd_cal_base_us = lc.hpd_coverage(cde_cal_us.detach().numpy(), y_grid_us, 
                                  y_cal_us.numpy())
hpd_test_base_us = lc.hpd_coverage(cde_test_us.detach().numpy(), y_grid_us, 
                                   y_test_us.numpy())

In [49]:
df_cs_group_cal_us = pd.DataFrame(data = {"x": x_cal_us.numpy().ravel(),
                                        "cs": hpd_cal_base_us.ravel(),
                                        "grouping":grouping_cal_us})



df_cs_group_test_us = pd.DataFrame(data = {"x": x_test_us.numpy().ravel(),
                                           "cs": hpd_test_base_us.ravel(),
                                           "grouping":grouping_test_us})

In [64]:
thresholds_hpd_test_us, _ = lc.thresholds_per_group(df_cs_group_cal_us,
                                                  desired_props = conformal_quantiles,
                                                  append = df_cs_group_test_us)

In [63]:
hpd_predict_grid_test_us = lc.hpd_grid(cde_test_us.detach().numpy(), 
                               z_delta = y_delta_us)

In [ ]:
#             #ipdb.set_trace()
#             v_iz, v_raw_iz, e_iz = lc.difference_validity_and_efficiency(
#                                          true_cde = true_cde_test_iz,
#                                          predict_grid = cde_test_iz,
#                                          true_grid = true_cde_test_iz,
#                                          thresholds_predict = thresholds_mat_test_iz,
#                                          thresholds_true = true_thresholds_test_iz,
#                                          df_cs_grouping = df_cs_group_test_iz,
#                                          true_grouping = g_test_iz.numpy().ravel(),
#                                          expected_prop = conformal_quantiles, 
#                                          z_delta = y_delta_iz,
#                                          verbose = False)
            

In [71]:
v_us, v_raw_us, e_us = lc.difference_validity_and_efficiency(
                                 true_cde = true_cde_test_us,
                                 predict_grid = hpd_predict_grid_test_us,
                                 true_grid = true_hpd_test_us,
                                 thresholds_predict = thresholds_hpd_test_us,
                                 thresholds_true = true_thresholds_test_us,
                                 df_cs_grouping = df_cs_group_test_us,
                                 true_grouping = g_test_us.numpy().ravel(),
                                 expected_prop = conformal_quantiles, 
                                 z_delta = y_delta_us,
                                 verbose = False)

In [72]:
info_left = pd.DataFrame(data = {"mdn_n_gaussian": np.array(["us"]),
                                   "mdn_lr": lr_1,
                                   "mdn_hidden1": hidden1_1,
                                   "mdn_hidden2": hidden2_1,
                                   "mdn_train_error": train_error_us.detach().numpy(),
                                   "mdn_val_error": val_error_us.detach().numpy(),
                                    "q_lr": lr_2,
                                   "q_hidden1": hidden1_2,
                                   "q_hidden2": hidden2_2,
                                   "q_train_error": q_train_error_us.detach().numpy(),
                                   "q_val_error": q_val_error_us.detach().numpy(),
                                   "k": k_val})

out = _combine_pivot_long_with_info(info_left, v_us, v_raw_us, e_us)



In [73]:
out.head()

,mdn_n_gaussian,mdn_lr,mdn_hidden1,mdn_hidden2,mdn_train_error,mdn_val_error,q_lr,q_hidden1,q_hidden2,q_train_error,q_val_error,k,grouping,quantile,means,type
0,us,0.001,10,10,3.497895,3.817615,0.001,10,10,28.782159,30.968704,3,0.0,0,0.982245,validity_oracle
1,us,0.001,10,10,3.497895,3.817615,0.001,10,10,28.782159,30.968704,3,1.0,0,0.996816,validity_oracle
2,us,0.001,10,10,3.497895,3.817615,0.001,10,10,28.782159,30.968704,3,2.0,0,0.957031,validity_oracle
3,us,0.001,10,10,3.497895,3.817615,0.001,10,10,28.782159,30.968704,3,3.0,0,0.997883,validity_oracle
4,us,0.001,10,10,3.497895,3.817615,0.001,10,10,28.782159,30.968704,3,4.0,0,0.960595,validity_oracle


In [82]:
def inner_process_us(data_all, k_range, n_gaussians_range, lr_range, 
                     hidden_range, verbose = True, 
                     epochs = 10000, 
                     y_range = np.array([-25,25]).reshape((-1,1)),
                     y_splits = 10000,
                     quantiles = np.arange(1,500)/500, #we are using these quantiles relative to conformal cuts... that isn't what was expected
                     conformal_quantiles = np.arange(1,500)/500,
                     t_range_size = 500):

    # data splitting ------
    data_us_list = lc.stratified_data_splitting(data_all, 
                                                prop_vec = np.array([.75,.75,.75,.75,1])/4)
    
    x_train_us,y_train_us, g_train_us = lc.torchify_data(data_us_list[0].x, 
                                        data_us_list[0].y, 
                                        data_us_list[0].group_info)
    x_val_us,y_val_us, g_val_us = lc.torchify_data(data_us_list[1].x, 
                                        data_us_list[1].y, 
                                        data_us_list[1].group_info)
    x_cde_cal_us, y_cde_cal_us, lc.torchify_data(data_us_list[2].x, 
                                        data_us_list[2].y, 
                                        data_us_list[2].group_info)
    x_cal_us,y_cal_us, g_cal_us = lc.torchify_data(data_us_list[3].x, 
                                        data_us_list[3].y, 
                                        data_us_list[3].group_info)
    x_test_us,y_test_us, g_test_us = lc.torchify_data(data_us_list[4].x, 
                                        data_us_list[4].y, 
                                        data_us_list[4].group_info)
    
    
    if verbose:
        bar = progressbar.ProgressBar()
        larger_iter = bar(list(itertools.product(n_gaussians_range, lr_range, hidden_range)))
    else:
        larger_iter = itertools.product(n_gaussians_range, lr_range, hidden_range)
    
    out_df_full = None
    
    y_grid_us = np.linspace(y_range[0], y_range[1],
                                y_splits).ravel()
    y_delta_us = (y_range[1]-y_range[0])/y_splits
    
    true_cde_test_us = lc.true_cde_out(x_test_us.numpy().ravel(), y_grid_us)
    true_hpd_test_us = lc.hpd_grid(true_cde_test_us, y_delta_us)
    
    # this is just the conformal scores since we're using HPD as the cs values
    true_thresholds_test_us = np.repeat(conformal_quantiles.reshape((1,-1)),
                                    x_test_us.shape[0], axis = 0)
    
    
    for n_guassian, lr_1, hidden_1 in \
        itertools.product(n_gaussians_range, lr_range, 
                          hidden_range): # no k_range and only MDN parameters currently

        hidden1_1 = hidden_1
        hidden2_1 = hidden_1
        # build MDN model (record training and validation loss) ------
        
        # us ---------
        mdn_model_us, mdn_opt_us, train_error_us, val_error_us = lc.tune_first_nn(
                                                         x_train = x_train_us,
                                                         y_train = y_train_us,
                                                         x_val = x_val_us,
                                                         y_val = y_val_us,
                                                         epochs = epochs,
                                                         n_gaussians = n_guassian,
                                                         n_hidden1 = hidden1_1,
                                                         n_hidden2 = hidden2_1,
                                                         lr = lr_1)
        mdn_model_us.prep_for_cde(y_range, y_splits)
        model_y_range_us = (mdn_model_us.y_range[0],
                         mdn_model_us.y_range[1])
        model_n_grid_us = mdn_model_us.n_grid
        y_grid_us = np.linspace(model_y_range_us[0], model_y_range_us[1], model_n_grid_us)
        
        # prepping for second model:
        cde_train_us = mdn_model_us.cde_predict_grid(x_train_us).detach().numpy()
        cde_val_us = mdn_model_us.cde_predict_grid(x_val_us).detach().numpy()
        
        hpd_train_base_us = lc.hpd_coverage(cde_train_us, y_grid_us, 
                                            y_train_us.numpy())
        hpd_val_base_us = lc.hpd_coverage(cde_val_us, y_grid_us, 
                                          y_val_us.numpy())
        
        hpd_train_us = torch.from_numpy(hpd_train_base_us.reshape((-1,1)))
        hpd_val_us = torch.from_numpy(hpd_val_base_us.reshape((-1,1)))
        
        # TODO: tune_first_nn & tune_second_nn: record training error too!
        
        for lr_2, hidden2 in itertools.product(lr_range, hidden_range): 
            # build Quantile Reg model (record training and validation loss) ---
            
            hidden1_2 = hidden2
            hidden2_1 = hidden2
            
            q_model, q_opt, q_train_error_us, q_val_error_us = lc.tune_second_nn(x_train = x_train_us,
                                                            y_train = hpd_train_us,
                                                            x_val = x_val_us,
                                                            y_val = hpd_val_us,
                                                            epochs = epochs,
                                                            quantiles = quantiles,
                                                            n_hidden1 = hidden1_2,
                                                            n_hidden2 = hidden2_2,
                                                            lr = lr_2,
                                                            verbose = False)
            
            hpd_estimate_cal_us = q_model(x_cal_us).detach().numpy()
            hpd_estimate_test_us = q_model(x_test_us).detach().numpy()
            
            # apply groupings  -----
            for k_val in k_range:
                kmeans_model_us, grouping_cal_us, _ = \
                    lc.profile_grouping(hpd_estimate_cal_us, k = k)

                _, _, grouping_test_us = lc.profile_grouping(
                                             profile_test = hpd_estimate_test_us, 
                                             Kmeans_model = kmeans_model_us)

                # hpd conformal scores... (only from MDN)
                cde_cal_us = mdn_model_us.cde_predict_grid(x_cal_us)
                cde_test_us = mdn_model_us.cde_predict_grid(x_test_us)


                hpd_cal_base_us = lc.hpd_coverage(cde_cal_us.detach().numpy(), y_grid_us, 
                                                  y_cal_us.numpy())
                hpd_test_base_us = lc.hpd_coverage(cde_test_us.detach().numpy(), y_grid_us, 
                                                   y_test_us.numpy())
                
                
                df_cs_group_cal_us = pd.DataFrame(data = {"x": x_cal_us.numpy().ravel(),
                                                        "cs": hpd_cal_base_us.ravel(),
                                                        "grouping":grouping_cal_us})

                df_cs_group_test_us = pd.DataFrame(data = {"x": x_test_us.numpy().ravel(),
                                                           "cs": hpd_test_base_us.ravel(),
                                                           "grouping":grouping_test_us})
            
            
                thresholds_hpd_test_us, _ = lc.thresholds_per_group(df_cs_group_cal_us,
                                                  desired_props = conformal_quantiles,
                                                  append = df_cs_group_test_us)
                
                hpd_predict_grid_test_us = lc.hpd_grid(cde_test_us.detach().numpy(), 
                                                        z_delta = y_delta_us)

                v_us, v_raw_us, e_us = lc.difference_validity_and_efficiency(
                                                 true_cde = true_cde_test_us,
                                                 predict_grid = hpd_predict_grid_test_us,
                                                 true_grid = true_hpd_test_us,
                                                 thresholds_predict = thresholds_hpd_test_us,
                                                 thresholds_true = true_thresholds_test_us,
                                                 df_cs_grouping = df_cs_group_test_us,
                                                 true_grouping = g_test_us.numpy().ravel(),
                                                 expected_prop = conformal_quantiles, 
                                                 z_delta = y_delta_us,
                                                 verbose = False)
        
                
                info_left = pd.DataFrame(data = {
                           "mdn_n_gaussian": np.array(["us"]),
                           "mdn_lr": lr_1,
                           "mdn_hidden1": hidden1_1,
                           "mdn_hidden2": hidden2_1,
                           "mdn_train_error": train_error_us.detach().numpy(),
                           "mdn_val_error": val_error_us.detach().numpy(),
                            "q_lr": lr_2,
                           "q_hidden1": hidden1_2,
                           "q_hidden2": hidden2_2,
                           "q_train_error": q_train_error_us.detach().numpy(),
                           "q_val_error": q_val_error_us.detach().numpy(),
                           "k": k_val})

                out = _combine_pivot_long_with_info(info_left, v_us, v_raw_us, e_us)

                if out_df_full is None:
                    out_df_full = out
                else:
                    out_df_full = pd.concat([out_df_full.reset_index(drop = True), 
                                             out.reset_index(drop = True)],
                                           axis = 0)
        
                return out_df_full


In [83]:
us_out = inner_process_us(data_all = data_all,
                      k_range = k_range_us[0:2], 
                      n_gaussians_range = n_guassians_range[0:2], 
                      lr_range = lr_range[0:2], 
                      hidden_range = hidden_range[0:2], 
                      verbose = True, 
                      epochs = 10000, 
                      y_range = np.array([-200,200]).reshape((-1,1)),
                      y_splits = 10000,
                      quantiles = np.arange(1,500)/500,
                      conformal_quantiles = np.arange(1,500)/500,
                      t_range_size = 500)

In [84]:
us_out

,mdn_n_gaussian,mdn_lr,mdn_hidden1,mdn_hidden2,mdn_train_error,mdn_val_error,q_lr,q_hidden1,q_hidden2,q_train_error,q_val_error,k,grouping,quantile,means,type
0,us,0.01,5,5,3.380025,3.725126,0.01,5,10,37.475128,37.176358,2,0.0,0,0.899216,validity_oracle
1,us,0.01,5,5,3.380025,3.725126,0.01,5,10,37.475128,37.176358,2,1.0,0,0.959414,validity_oracle
2,us,0.01,5,5,3.380025,3.725126,0.01,5,10,37.475128,37.176358,2,2.0,0,0.960224,validity_oracle
3,us,0.01,5,5,3.380025,3.725126,0.01,5,10,37.475128,37.176358,2,3.0,0,0.962655,validity_oracle
4,us,0.01,5,5,3.380025,3.725126,0.01,5,10,37.475128,37.176358,2,4.0,0,0.853615,validity_oracle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7979,us,0.01,5,5,3.380025,3.725126,0.01,5,10,37.475128,37.176358,2,11.0,498,399.733333,efficiency_oracle
7980,us,0.01,5,5,3.380025,3.725126,0.01,5,10,37.475128,37.176358,2,12.0,498,398.978000,efficiency_oracle
7981,us,0.01,5,5,3.380025,3.725126,0.01,5,10,37.475128,37.176358,2,13.0,498,398.755200,efficiency_oracle
7982,us,0.01,5,5,3.380025,3.725126,0.01,5,10,37.475128,37.176358,2,14.0,498,399.200000,efficiency_oracle
